# Text Classification with 1-d CNN Encoding and LSTM

이전 실습들을 통해 LSTM 등의 RNN을 이용하여 Text를 state vector로 encoding하거나, 혹은 1-d CNN을 이용하여 encoding하여, 이를 Feature 로 활용하여 Text classification하는 IMDB dataset 기반 sentimental analysis task를 다루어 보았습니다.

이번 실습처럼 500 이상의 시퀀스 길이를 가지는 경우 RNN의 학습 속도가 너무 느리다는 문제가 있습니다. CNN을 사용하면 GPU의 병렬처리 연산 효율을 극대화하여 학습 속도가 매우 빠르면서도 정확도가 꽤 높은 결과를 확인할 수 있었습니다. 그러나, 수백개의 단어 및 10개 이상의 문장으로 이루어진 긴 글을 분석함에 있어서 CNN만 사용했을 때 state 변화 개념이 들어가지 않은 채 특정 키워드의 분포만으로 결론을 단정짓게 되는 구조적인 약점 또한 존재합니다.  

만약 그렇다면, CNN과 RNN의 장점을 결합한 모델을 생각해 볼 수는 없을까요? 긴 길이의 입력 시퀀스를 먼저 1-d CNN을 사용하여 보다 짧은 길이의 시퀀스로 인코딩하여 변환한 후 다시 RNN을 적용하는 방식으로 동일한 task를 다시 다루어 보도록 하겠습니다.

(참고)  
https://github.com/gilbutITbook/006975/blob/master/6.4-sequence-processing-with-convnets.ipynb  

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
# import imdb
from keras.datasets import imdb

# 데이터 관련 설정은 LSTM 케이스와 동일하게 한다.
max_features = 10000
max_tokens = 580
embedding_size = 8

# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)   # 원래는 이 라인만 있으면 된다.
# restore np.load for future normal usage
np.load = np_load_old

print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

Using TensorFlow backend.


Train-set size:  25000
Test-set size:   25000


In [4]:
pad = 'pre'
x_train_pad = pad_sequences(x_train, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape

(25000, 580)

## Create Model with Conv1D and GRU

In [7]:
model = Sequential()
model.add(Embedding(input_dim=max_features,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))

#########################################
# model.add(GlobalMaxPooling1D())
model.add(GRU(units=32))    # 03.02.01과 이 한줄만 다름
#########################################

model.add(Dense(1))

model.summary()

model.compile(optimizer=Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 580, 8)            80000     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 574, 32)           1824      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 114, 32)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 108, 32)           7200      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 21, 32)            0         
_________________________________________________________________
gru (GRU)                    (None, 32)                6240      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total para

In [8]:
layer_outputs = [layer.output for layer in model.layers]
layer_outputs

[<tf.Tensor 'layer_embedding_2/embedding_lookup/Identity_1:0' shape=(?, 580, 8) dtype=float32>,
 <tf.Tensor 'conv1d_5/Relu:0' shape=(?, 574, 32) dtype=float32>,
 <tf.Tensor 'max_pooling1d_4/Squeeze:0' shape=(?, 114, 32) dtype=float32>,
 <tf.Tensor 'conv1d_6/Relu:0' shape=(?, 108, 32) dtype=float32>,
 <tf.Tensor 'max_pooling1d_5/Squeeze:0' shape=(?, 21, 32) dtype=float32>,
 <tf.Tensor 'gru/strided_slice_12:0' shape=(?, 32) dtype=float32>,
 <tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 1) dtype=float32>]

In [9]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=100, batch_size=64)

Train on 23750 samples, validate on 1250 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
23750/23750 [==============================] - 19s 784us/sample - loss: 1.7284 - acc: 0.5000 - val_loss: 1.4043 - val_acc: 0.5008
Epoch 2/100
23750/23750 [==============================] - 16s 694us/sample - loss: 1.2608 - acc: 0.5000 - val_loss: 1.1291 - val_acc: 0.5008
Epoch 3/100
23750/23750 [==============================] - 17s 703us/sample - loss: 1.0329 - acc: 0.5000 - val_loss: 0.9454 - val_acc: 0.5008
Epoch 4/100
23750/23750 [==============================] - 17s 713us/sample - loss: 0.8792 - acc: 0.5000 - val_loss: 0.8178 - val_acc: 0.5008
Epoch 5/100
23750/23750 [==============================] - 17s 704us/sample - loss: 0.7766 - acc: 0.5000 - val_loss: 0.7418 - val_acc: 0.5008
Epoch 6/100
23750/23750 [==============================] - 18s 748us/sample - loss: 0.7219 - acc: 0.5000 - val_loss: 0.7074 - val_acc: 0.5008
Epoch 7/100
23750/23750 [==========================

KeyboardInterrupt: 

In [ ]:
result = model.evaluate(x_test_pad, y_test)

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))